In [11]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [12]:
file_path = r"C:\Users\advait\Desktop\converted_distillation_data.csv"
df = pd.read_csv(file_path)
inputs = ['Feed_Flow_Rate', 'Feed_Composition', 'Reflux_Ratio', 'Boil_Up_Ratio']
X = df[inputs]

In [13]:
rf_model = joblib.load('rf_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')

In [19]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
X_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv')

In [21]:
train_pred1 = rf_model.predict(X_train)
train_pred2 = xgb_model.predict(X_train)

In [23]:
train_meta = np.column_stack((train_pred1, train_pred2))
meta_model = LinearRegression()
meta_model.fit(train_meta, y_train)

LinearRegression()

In [72]:
def approximate_energy_efficiency(Q_R, Q_C, efficiency_factor=0.3):

    Q_useful = Q_R * efficiency_factor
    total_energy_input = Q_R + Q_C

    if total_energy_input == 0: 
        return 0

    efficiency = (Q_useful / total_energy_input) * 100
    return efficiency

In [78]:
# Declare global variables for individual parameters
Feed_Flow_Rate = 0
Feed_Composition = 0
Reflux_Ratio = 0
Boil_Up_Ratio = 0

def predict_from_user_input():
    global user_input, Feed_Flow_Rate, Feed_Composition, Reflux_Ratio, Boil_Up_Ratio

    # Take user input and assign to global variables
    Feed_Flow_Rate = float(input("Enter Feed Flow Rate: "))
    Feed_Composition = float(input("Enter Feed Composition: "))
    Reflux_Ratio = float(input("Enter Reflux Ratio: "))
    Boil_Up_Ratio = float(input("Enter Boil-Up Ratio: "))

    # Use the inputs to create the DataFrame
    user_input = pd.DataFrame({
        'Feed_Flow_Rate': [Feed_Flow_Rate],
        'Feed_Composition': [Feed_Composition],
        'Reflux_Ratio': [Reflux_Ratio],
        'Boil_Up_Ratio': [Boil_Up_Ratio],
    })

    pred1 = rf_model.predict(user_input)
    pred2 = xgb_model.predict(user_input)

    meta_input = np.column_stack((pred1, pred2))
    final_prediction = meta_model.predict(meta_input)

    print("Predicted Output:", final_prediction)

# Call the function
predict_from_user_input()

# Use the updated global variables in the next cell or anywhere in the code
Distillate_Flow_Rate = Feed_Flow_Rate / (1 + Boil_Up_Ratio)
Bottoms_Flow_Rate = Feed_Flow_Rate - Distillate_Flow_Rate

print("Distillate Flow Rate:", Distillate_Flow_Rate)
print("Bottoms Flow Rate:", Bottoms_Flow_Rate)


Enter Feed Flow Rate:  78.5296
Enter Feed Composition:  25.47046861
Enter Reflux Ratio:  6.181017057
Enter Boil-Up Ratio:  0.123429044


Predicted Output: [[ 32.50272828 -31.50272828 266.58103362 142.35747226]]
Distillate Flow Rate: 69.90169999557176
Bottoms Flow Rate: 8.627900004428241


In [80]:
def compute_feed_purity(feed_flow, distillate_flow, bottoms_flow, distillate_purity, bottoms_purity):

    if feed_flow == 0:  # Avoid division by zero
        return 0

    feed_purity = (distillate_flow * distillate_purity + bottoms_flow * bottoms_purity) / feed_flow
    return feed_purity


feed_flow = Feed_Flow_Rate
distillate_flow = Distillate_Flow_Rate
bottoms_flow = Bottoms_Flow_Rate
distillate_purity = 0.95 
bottoms_purity = 0.05 

feed_purity = compute_feed_purity(feed_flow, distillate_flow, bottoms_flow, distillate_purity, bottoms_purity)
print(f"Overall Feed Purity: {feed_purity:.4f} (mole fraction)")

Overall Feed Purity: 0.8511 (mole fraction)
